# 문제 1. CNN으로 MNIST 분류해보기

### 필요한 모듈 가져오기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.modules.dropout import Dropout2d
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
# CPU 혹은 GPU 장치 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 1.  MNIST 데이터셋 내려받기
**문제 1-1** 코드 빈칸을 채워주세요. **(1점)**

In [ ]:
transform = transforms.ToTensor() # 텐서로 변환

In [ ]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST', # 다운로드 경로 지정
    train = True, # True를 지정하면 훈련 데이터로 다운로드
    download = True,
    transform = transform)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transform)

## 2. 데이터를 데이터로더에 전달

**문제 1-2**
코드 빈칸을 채워주세요. **(1점)**
<br><br>(배치사이즈 = 100)

In [ ]:
train_loader = DataLoader(train_set, batch_size = 100, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 100, shuffle = True)

## 3. input_size 확인

In [ ]:
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

## 4. 합성곱 네트워크 생성

**문제 1-3** 코드 빈칸을 채워주세요. **(3점)**

In [ ]:
class CNN(nn.Module):
    def __init__(self):
      super(CNN, self).__init__()
      # 첫번째층
      # 입력이미지 shape (?, 28, 28, 1)
      # conv (?, 28, 28, 32)
      # pooling (?, 14, 14, 32)

      self.layer1 = torch.nn.Sequential(
          torch.nn.Conv2d(1, 32, kernel_size =3 , stride=1, padding=1),
          nn.BatchNorm2d(32),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # 두번째층
      # 입력이미지 shape (?, 14, 14, 32)
      # conv (?, 14, 14, 64)
      # pooling (?, 7, 7, 64)

      self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32,64, kernel_size =3 , stride=1, padding=1),
          nn.BatchNorm2d(64),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

      # 7*7*64 inputs -> 10 Outputs
      self.fc1 = torch.nn.Linear(7*7*64, 600, bias = True)
      self.drop = nn.Dropout2d(0.25)
      self.fc2 = torch.nn.Linear(600, 120, bias = True)
      self.fc3 = torch.nn.Linear(120, 10, bias = True)

    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.view(out.size(0), -1) # Flatten
      out = self.fc1(out)
      out = self.drop(out)
      out = self.fc2(out)
      out = self.fc3(out)
      return out



## 5. 합성곱 네트워크를 위한 모델 및 파라미터 정의

**문제 1-4** 코드 빈칸을 채워주세요. **(1점)**

In [ ]:
learning_rate = 0.001
num_epochs = 5

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

# 비용 함수와 optimizer 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

print(model) # 모델 확인

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


## 6. 모델 학습 및 성능 평가

**문제 1-5** 아래 코드 빈칸을 자유롭게 채워주세요. **(4점)**
<br><br> (학습 데이터를 이용하여 모델을 학습시키고, test_set으로 모델 성능을 평가해주세요.)

In [ ]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []


for epoch in range(num_epochs):

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    train = Variable(images.view(100,1,28,28))
    labels = Variable(labels)

    outputs = model(train)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    count += 1

    #test
    if not (count%50):
      total = 0
      correct = 0
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_list.append(labels)
        test = Variable(images.view(100,1,28,28))
        outputs = model(test)
        predictions = torch.max(outputs, 1)[1].to(device)
        predictions_list.append(predictions)
        correct +=  (predictions == labels).sum()
        total += len(labels)

      accuracy = correct * 100 / total
      loss_list.append(loss.data)
      iteration_list.append(count)
      accuracy_list.append(accuracy)

    if not(count%500):
      print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data,
                                                            accuracy))

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Iteration: 500, Loss: 0.0922522246837616, Accuracy: 97.81999969482422%
Iteration: 1000, Loss: 0.17655937373638153, Accuracy: 97.6500015258789%
Iteration: 1500, Loss: 0.047286368906497955, Accuracy: 98.20999908447266%
Iteration: 2000, Loss: 0.015769969671964645, Accuracy: 98.29000091552734%
Iteration: 2500, Loss: 0.009886989369988441, Accuracy: 98.69000244140625%
Iteration: 3000, Loss: 0.23679059743881226, Accuracy: 98.75%
Iteration: 3500, Loss: 0.026725171133875847, Accuracy: 98.13999938964844%
Iteration: 4000, Loss: 0.011718673631548882, Accuracy: 98.5999984741211%
Iteration: 4500, Loss: 0.023680973798036575, Accuracy: 98.95999908447266%
Iteration: 5000, Loss: 0.001206500455737114, Accuracy: 98.41999816894531%
Iteration: 5500, Loss: 0.007774607744067907, Accuracy: 98.81999969482422%
Iteration: 6000, Loss: 0.1092161163687706, Accuracy: 98.91000366210938%


# 문제2
CIFAR10 데이터 불러오기 (torchvision.datasets)

In [ ]:
import os
import time
import copy
import glob
import cv2
import shutil

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt


# Hyperparameters
batch_size = 64

# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))]
    )

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:13<00:00, 12895637.00it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
print(len(trainset), len(testset))

50000 10000


In [ ]:
next(iter(trainloader))[0].shape

torch.Size([64, 3, 32, 32])

전이 학습

모델 객체 생성(VGG16 모델 사용)

In [ ]:
# Define VGG model
model = models.vgg16(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:26<00:00, 20.5MB/s]


### Q1. 모델의 가중치를 전부 고정하시오.(1점)

In [ ]:
for param in model.parameters():
  param.requires_grad = False

### Q2. 모델에서 데이터 분류기(classifier)의 마지막 layer를 CIFAR10 데이터의 분류 클래스 수에 맞게 조정하시오.(1점)

In [ ]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
print(model.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)


In [ ]:
model.classifier[6] = nn.Linear(4096, 10)

### Q3. 데이터 분류기(classifier) 층의 가중치 고정을 해제하시오. (1점)

In [ ]:
for param in model.classifier.parameters():
  param.requires_grad = True

### Q4. 손실함수 및 optimizer를 정의하시오. (optimizer : Adam사용, lr=0.001) (2점)


In [ ]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params_to_update,  lr=0.001)

### Q5. 모델을 학습하시오. (num_epochs = 10) (3점)

(학습시간을 짧게하기 위해 데이터 증강 및 이미지 크기 조정 등을 하지 않았고 이에 따라 성능이 구린게 맞습니다.)(gpu사용 시 약 10분 소요)

(매 epoch마다 train_loss와 train_accuracy를 출력할것)

(매 epoch마다 모델을 저장할 것)

In [ ]:
# Training loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = 10

train_acc_history = []
train_loss_history = []
train_best_acc = 0.0

for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        model.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = running_corrects.double() / len(trainloader.dataset)

    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    if epoch_acc > train_best_acc:
        train_best_acc = epoch_acc

    train_acc_history.append(epoch_acc.item())
    train_loss_history.append(epoch_loss)
    os.makedirs(os.path.join('model'), exist_ok=True)
    torch.save(model.state_dict(), os.path.join('model/', '{0:0=2d}.pth'.format(epoch)))
    print()

print('Best Acc: {:4f}'.format(train_best_acc))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0/9
----------
Loss: 1.5395 Acc: 0.5194


 10%|█         | 1/10 [01:06<10:02, 66.97s/it]


Epoch 1/9
----------
Loss: 1.4383 Acc: 0.5438


 20%|██        | 2/10 [01:58<07:42, 57.86s/it]

...


Epoch 8/9
----------
Loss: 1.2460 Acc: 0.6010


 90%|█████████ | 9/10 [08:15<00:54, 54.43s/it]


Epoch 9/9
----------
Loss: 1.2312 Acc: 0.6043


100%|██████████| 10/10 [09:07<00:00, 54.77s/it]


Best Acc: 0.604340


### Q6. 저장했던 10개의 모델을 불러와 테스트 데이터를 평가하시오.(2점)

(매 epoch마다 test_accuracy를 출력할 것)



In [ ]:
test_acc_history = []
test_best_acc = 0.0

saved_models = glob.glob('model/' + '*.pth')
saved_models.sort()
print('saved_model', saved_models)

for model_path in saved_models:
    print('Loading model', model_path)

    model.load_state_dict(torch.load(model_path))
    model.eval()
    model.to(device)
    running_corrects = 0

    for inputs, labels in testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)

        _, preds = torch.max(outputs, 1)
        running_corrects += preds.eq(labels).int().sum()

    epoch_acc = running_corrects.double() / len(testloader.dataset)
    print('Acc: {:.4f}'.format(epoch_acc))

    if epoch_acc > test_best_acc:
        test_best_acc = epoch_acc

    test_acc_history.append(epoch_acc.item())
    print()

print('Best Acc: {:4f}'.format(test_best_acc))

saved_model ['model/00.pth', 'model/01.pth', 'model/02.pth', 'model/03.pth', 'model/04.pth', 'model/05.pth', 'model/06.pth', 'model/07.pth', 'model/08.pth', 'model/09.pth']
Loading model model/00.pth
Acc: 0.6200

Loading model model/01.pth
Acc: 0.6171

Loading model model/02.pth
Acc: 0.6376

Loading model model/03.pth
Acc: 0.6239

Loading model model/04.pth
Acc: 0.6325

Loading model model/05.pth
Acc: 0.6375

Loading model model/06.pth
Acc: 0.6341

Loading model model/07.pth
Acc: 0.6324

Loading model model/08.pth
Acc: 0.6528

Loading model model/09.pth
Acc: 0.6501

Best Acc: 0.652800
